In [5]:
import numpy as np
from tqdm.notebook import tqdm
from skimage import io, transform
import glob
import argparse
import matplotlib.pyplot as plt
import sys
import os
from google.colab import drive
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/MyDrive/OS-SIM/')
from SIMulator_functions import *

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [6]:
# ------------ Parameters-------------
def GetParams():
    opt = argparse.Namespace()

    # phase shifts for each stripe
    opt.Nshifts = 3
    # number of orientations of stripes
    opt.Nangles = 1
    # modulation factor
    opt.ModFac = 0.6+0.35*np.random.rand()
    # orientation offset
    opt.alpha = -68*np.pi/180
    # orientation error
    opt.angleError = (0.5-np.random.rand())*2*np.pi
    # shuffle the order of orientations
    opt.shuffleOrientations = False
    # random phase shift errors
    opt.phaseError = pi*(0.5-np.random.rand(opt.Nangles, opt.Nshifts))
    # in percentage
    opt.NoiseLevel = 18 + 8*(np.random.rand()-0.5)
    # Poisson noise extent
    # include OTF and GT in stack
    opt.Poisson = np.random.randint(80,5000)
    opt.OTF_and_GT = True
    # NA
    opt.NA = 1.2
    # Emission wavelength
    opt.emission = np.random.randint(500,690)
    # Pattern frequencies  
    kMax = (2*opt.NA)/(opt.emission)
    factor = 0.7+0.3*np.random.rand() # Adjust for effects of aberrations
    opt.offset = factor
    opt.k2 = (0.4+np.random.rand()*0.15)*kMax*factor
    # Pixel Size
    opt.Psize = np.random.randint(75,90)
    # Image to use for ground truth in training
    opt.target = 'original' # 'original' or 'SIM' 
    # Location to save test images
    opt.sLoc = '/content/gdrive/MyDrive/training data/generated 27-03-2021'
    # Out of focus depth
    opt.depthO = 700+800*np.random.rand()
    
    return opt


In [7]:
files = glob.glob('/content/gdrive/MyDrive/DIV2K_train_HR/*.png')


In [8]:
# ------------ Options --------------
opt = GetParams()
# ------------ Run --------------
Io = io.imread(files[0]) / 255
Io = Io - 0.3*np.amax(Io)
Io[Io<0] = 0

os.makedirs(opt.sLoc,exist_ok=True)

n_rep = 3
sNum = 2030

for n_it in tqdm(range(n_rep),desc='Iterations done'):
    n_it = n_it+1
    # ------------ Main loop --------------
    for f in tqdm(range(len(files)),desc='Images done'):
        
        type = np.random.rand()
        if type > 0.05:
          # In focus image
          file = files[f]     
          Io = io.imread(file) 
          Io = Io / np.amax(Io)
          Io = Io - 0.35 # Cropping LUT like this increases the number of sharp details in the image 
          Io[Io<0] = 0
          Io = Io/np.amax(Io)
          
          minDim = np.amin(Io[:,:,0].shape)
          # This is a very ugly way of re-sampling regions of the image
          Io = np.rot90(Io,np.random.randint(0,3))
          Io = Io[0:minDim,0:minDim,:]

          Io = transform.resize(Io, (512, 512), anti_aliasing=True)
          if  Io.shape[2] > 1:
                      Io = Io.mean(2)

        
        else:
          Io =np.zeros([512,512])
          row,col = Io.shape
          s_vs_p = 0.5
          amount = np.random.rand()*0.1
          # Salt mode
          num_salt = np.ceil(amount * Io.size * s_vs_p)
          coords = [np.random.randint(0, i - 1, int(num_salt))
                  for i in Io.shape]
          Io[coords] = 1
                  

        

       # Out of focus image
        fnew = files[np.random.randint(0, len(files))]
        Oi = io.imread(fnew) 
        Oi = Oi / np.amax(Oi)
        Oi = Oi - 0.3
        Oi[Oi<0] = 0

        minDim = np.amin(Oi[:,:,0].shape)
        Oi = np.rot90(Oi,np.random.randint(0,3))
        Oi = Oi[0:minDim-1,0:minDim-1,:]
        Oi = transform.resize(Oi, (512, 512), anti_aliasing=True)

        if  Oi.shape[2] > 1: # if not grayscale
            Oi = Oi.mean(2)  
            Oi = Oi/np.amax(Oi)

        opt = GetParams()
        stack = Generate_SIM_Image(opt, Io, Oi)

        # normalise
        for i in range(len(stack)):
                stack[i,:,:] = stack[i,:,:]-np.amin(stack[i,:,:])
                stack[i,:,:] = 255*stack[i,:,:]/np.amax(stack[i,:,:])       
                
        stack = stack.astype('uint8')
        svPath = os.path.join(opt.sLoc, '%d.tif' % sNum)
        io.imsave(svPath,stack)
        
      
            
        sNum += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


# New section